In [76]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

from datetime import date
import pandas as pd
date_today = date.today()

SCROLL_PAUSE_TIME = 3

URL = 'https://www.sergiotacchini.com/collections/womens-new-arrivals'
#URL = 'https://www.sergiotacchini.com/collections/mens-new-arrivals'
driver = webdriver.Chrome('/Users/mandili/.wdm/drivers/chromedriver/87.0.4280.20/mac64/chromedriver')
driver.get(URL)

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

soup = BeautifulSoup(driver.page_source,'html.parser')

In [77]:
product_url = []
picture_url = []
name = []
color = []
timestamp = []
brand = []
product_type = []
price = []
sales_price = []

for i in soup.find_all('div',{'class':'product-item small--one-half medium--one-half large-up--one-quarter'}):
    
    timestamp.append(date_today)
    brand.append('Sergio Tacchini')
    product_type.append("Women's Clothing")
    
    product_url.append("https://www.sergiotacchini.com" + i.a.get('href'))
    index = str(i.find('div',{'class':'product-item__bg aos-init aos-animate lazyloaded'}).get('data-bgset')).find('180w 240h,')
    picture_url.append("https:" + str(i.find('div',{'class':'product-item__bg aos-init aos-animate lazyloaded'}).get('data-bgset'))[:index].strip().replace('180x','600x'))
    
    color_index = str(i.find('h3',{'class':'title'}).text).find(' - ')
    name.append(str(i.find('h3',{'class':'title'}).text)[:color_index])
    color.append(str(i.find('h3',{'class':'title'}).text.lower())[color_index+3:])
    
    if i.find('span',{'class':'old-price'}) != None:
        price.append(i.find('span',{'class':'old-price'}).text)
        price_index = str(i.find('span',{'class':'price sale'}).text).find('USD')
        sales_price.append(str(i.find('span',{'class':'price sale'}).text)[price_index+3:].strip())
    else:
        price.append(None)
        sales_price.append(i.find('span',{'class':'price'}).text.strip())
        
    

# Second Layer

In [78]:
product_description = []
material = []
picture_name = []
counter = 1

for link in product_url:
    item_color = []
    ourUrl = urllib.request.urlopen(link)
    soup = BeautifulSoup(ourUrl,'html.parser')
    
    picture_name.append('sergio-tacchini-women_' + str(counter) + '.jpg') # change variable
    counter += 1
    
    product_description.append(soup.find('div',{'class':'product-description rte'}).p.text)
    
    item_mat = []
    for material_i in soup.find('div',{'class':'product-description rte'}).ul.find_all('li'):
        item_mat.append(material_i.text.replace('\n','').replace('\xa0',''))
    material.append(item_mat)

In [79]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp", "picture_name"]
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = name
second_pass.product_description = product_description
second_pass.price = price
second_pass.sales_price = sales_price
second_pass.color = color
second_pass.material = material
second_pass.product_url = product_url
second_pass.picture_url = picture_url
second_pass.timestamp = timestamp
second_pass.picture_name = picture_name
second_pass.head()

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp,picture_name
0,Sergio Tacchini,Women's Clothing,Ella Sweatpants,A performance-inspired update to traditional j...,65.00 USD,48.75 USD,navy/white,"[Unbrushed fleece pant, 80/20 cotton-poly 300g...",https://www.sergiotacchini.com/collections/wom...,https://cdn.shopify.com/s/files/1/0153/4230/53...,2020-12-07,sergio-tacchini-women_1.jpg
1,Sergio Tacchini,Women's Clothing,Ella Full Zip Hoodi,"An on and off court essential, the Ella Full Z...",75.00 USD,56.25 USD,la full zip hoodie,"[Zip-up hoodie with metal zipper pull, 80/20 c...",https://www.sergiotacchini.com/collections/wom...,https://cdn.shopify.com/s/files/1/0153/4230/53...,2020-12-07,sergio-tacchini-women_2.jpg
2,Sergio Tacchini,Women's Clothing,Ella Sweatpants,A performance-inspired update to traditional j...,65.00 USD,48.75 USD,dark grey melange/navy,"[Unbrushed fleece pant, 80/20 cotton-poly 300g...",https://www.sergiotacchini.com/collections/wom...,https://cdn.shopify.com/s/files/1/0153/4230/53...,2020-12-07,sergio-tacchini-women_3.jpg
3,Sergio Tacchini,Women's Clothing,Ella Shorts,Crafted from super-comfortable stretch jersey ...,55.00 USD,41.25 USD,navy/white,"[Jersey spandex stretch tennis shorts, Sergio ...",https://www.sergiotacchini.com/collections/wom...,https://cdn.shopify.com/s/files/1/0153/4230/53...,2020-12-07,sergio-tacchini-women_4.jpg
4,Sergio Tacchini,Women's Clothing,Ella Shorts,Crafted from super-comfortable stretch jersey ...,55.00 USD,41.25 USD,white/red,"[Jersey spandex stretch tennis shorts, Sergio ...",https://www.sergiotacchini.com/collections/wom...,https://cdn.shopify.com/s/files/1/0153/4230/53...,2020-12-07,sergio-tacchini-women_5.jpg


In [80]:
second_pass.to_csv("sergio-tacchini-women.csv", index=False, encoding = 'utf-8-sig')

# Download Images

In [81]:
import os
import requests as req

new_pd = pd.read_csv('/Users/mandili/Desktop/sergio-tacchini-women.csv', encoding='utf-8-sig')

os.mkdir('sergio-tacchini-women')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    img_data = req.get(link).content
    with open('sergio-tacchini-women/'+ "sergio-tacchini-women_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1